# 测试base完整模型的性能

## 载入数据和模型

In [1]:
from datasets import load_dataset

dataset = load_dataset(path="imagefolder", data_dir="E:/jupyter/VIT_example/dataset/hfdataset_Mini") #路径不能有中文

Resolving data files:   0%|          | 0/5380 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/312 [00:00<?, ?it/s]

In [28]:

from transformers import AutoImageProcessor

# 加载图像处理器
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")

def transforms(examples):
    images = [img.convert("RGB").resize((224, 224)) for img in examples["image"]]  # 图片被转换为RGB通道，同时缩放至224*224
    
    examples["pixel_values"] = image_processor(images, return_tensors="pt")["pixel_values"] # 使用image_processor处理图像，生成pixel_values（张量图像）
    return examples

dataset.set_transform(transforms)
dataset['train'][0].keys()

import torch

def collate_fn(batch): # batch应包含pixel_values和labels
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]), # 返回堆叠的图像张量
        'labels': torch.tensor([x['labels'] for x in batch]) # 返回堆叠的标签张量
    }

import numpy as np
import evaluate

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(p):
    # 计算预测结果
    predictions = np.argmax(p.predictions, axis=1)

    # 计算各个指标
    accuracy = accuracy_metric.compute(predictions=predictions, references=p.label_ids)
    f1 = f1_metric.compute(predictions=predictions, references=p.label_ids, average='weighted')
    precision = precision_metric.compute(predictions=predictions, references=p.label_ids, average='weighted')
    recall = recall_metric.compute(predictions=predictions, references=p.label_ids, average='weighted')

    # 输出结果
    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1'],
        'precision': precision['precision'],
        'recall': recall['recall']
    }


for split in dataset:
    dataset[split] = dataset[split].rename_column('label', 'labels') # 将 'label' 特征名改为 'labels'

ValueError: Original column name label not in the dataset. Current columns in the dataset: ['image', 'labels']

In [6]:
from transformers import ViTForImageClassification

model_name = 'google/vit-base-patch16-224-in21k' # 指定VIT模型

In [7]:
model = ViTForImageClassification.from_pretrained("./vit-base-covid")

In [29]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-covid", # 模型保存的目录
  per_device_train_batch_size=16, # 越大模型越稳定但需要消耗更大显存
  evaluation_strategy="steps", #模型评估策略，steps表示每过一定次数评估一次模型
  num_train_epochs=100, #训练轮数
  fp16=True, #半精度浮点，可减少内存占用，需要设备支持
  save_steps=100, #多少步保存一次模型
  eval_steps=100, #多少步评估一次模型
  logging_steps=3, #多少步评估一次日志
  learning_rate=2e-4, #学习率 越低越容易过拟合，越高收敛越慢
  save_total_limit=10, #保存的模型总数
  remove_unused_columns=False, #删除未使用列，减少内存消耗
  push_to_hub=False, #是否将模型发布到hf社区
  report_to='tensorboard', #日志的报告地
  load_best_model_at_end=True, #是否自动加载最优模型
  ignore_data_skip=True #从断点继续训练模型
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=image_processor,
    
)


## 基准测试

In [30]:
metrics = trainer.evaluate(dataset['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)


***** eval metrics *****
  eval_accuracy           =     0.8301
  eval_f1                 =     0.8323
  eval_loss               =     0.6882
  eval_precision          =     0.8767
  eval_recall             =     0.8301
  eval_runtime            = 0:00:11.53
  eval_samples_per_second =      27.04
  eval_steps_per_second   =       3.38


C:\Users\w1586\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 外部验证

In [1]:
from datasets import load_dataset

Outerdataset = load_dataset(path="imagefolder", data_dir="E:/jupyter/VIT_example/dataset/Outerdataset") #路径不能有中文
Outerdataset.set_transform(transforms)
Outerdataset['test'][0].keys()
for split in Outerdataset:
    Outerdataset[split] = Outerdataset[split].rename_column('label', 'labels') # 将 'label' 特征名改为 'labels'

Resolving data files:   0%|          | 0/312 [00:00<?, ?it/s]

In [ ]:
metrics = trainer.evaluate(Outerdataset["test"])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

## 对特定图片进行预测

In [11]:
num=1
image=dataset["test"][num]["image"]
dataset["test"][num]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1458x1303>,
 'labels': 0,
 'pixel_values': tensor([[[-0.6235, -0.6157, -0.6078,  ..., -0.9843, -0.9843, -0.9843],
          [-0.5843, -0.5765, -0.5686,  ..., -0.9843, -0.9843, -0.9843],
          [-0.5608, -0.5686, -0.5608,  ..., -0.9843, -0.9843, -0.9843],
          ...,
          [-0.1059, -0.0824, -0.0431,  ..., -0.3333, -0.3647, -0.3804],
          [-0.1137, -0.0824, -0.0353,  ..., -0.3412, -0.3647, -0.3882],
          [-0.1137, -0.0824, -0.0353,  ..., -0.3490, -0.3725, -0.3882]],
 
         [[-0.6235, -0.6157, -0.6078,  ..., -0.9843, -0.9843, -0.9843],
          [-0.5843, -0.5765, -0.5686,  ..., -0.9843, -0.9843, -0.9843],
          [-0.5608, -0.5686, -0.5608,  ..., -0.9843, -0.9843, -0.9843],
          ...,
          [-0.1059, -0.0824, -0.0431,  ..., -0.3333, -0.3647, -0.3804],
          [-0.1137, -0.0824, -0.0353,  ..., -0.3412, -0.3647, -0.3882],
          [-0.1137, -0.0824, -0.0353,  ..., -0.3490, -0.3725, -0.3882

In [12]:
from transformers import pipeline

classifier = pipeline("image-classification", model='./vit-base-covid/' )
classifier(image)

[{'score': 0.9979442954063416, 'label': 'Covid'},
 {'score': 0.0007841807091608644, 'label': 'Lung Opacity'},
 {'score': 0.0006628449191339314, 'label': 'Normal'},
 {'score': 0.0006086361245252192, 'label': 'Viral Pneumonia'}]

## 进行Covid与非Covid的二分类预测

In [13]:
def compute_metrics(p):
    # 将预测结果中的所有非0（非Covid）标签转换为1
    predictions = np.argmax(p.predictions, axis=1)
    binary_predictions = np.where(predictions == 0, 0, 1)

    # 将真实标签中的所有非0（非Covid）标签转换为1
    binary_references = np.where(p.label_ids == 0, 0, 1)

    # 计算二分类的各个指标
    accuracy = accuracy_metric.compute(predictions=binary_predictions, references=binary_references)['accuracy']
    f1 = f1_metric.compute(predictions=binary_predictions, references=binary_references, average='binary')['f1']
    precision = precision_metric.compute(predictions=binary_predictions, references=binary_references, average='binary')['precision']
    recall = recall_metric.compute(predictions=binary_predictions, references=binary_references, average='binary')['recall']

    # 返回计算的指标
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-simple-covid", # 模型保存的目录
  per_device_train_batch_size=16, # 越大模型越稳定但需要消耗更大显存
  evaluation_strategy="steps", #模型评估策略，steps表示每过一定次数评估一次模型
  num_train_epochs=100, #训练轮数
  fp16=True, #半精度浮点，可减少内存占用，需要设备支持
  save_steps=100, #多少步保存一次模型
  eval_steps=100, #多少步评估一次模型
  logging_steps=3, #多少步评估一次日志
  learning_rate=2e-4, #学习率 越低越容易过拟合，越高收敛越慢
  save_total_limit=10, #保存的模型总数
  remove_unused_columns=False, #删除未使用列，减少内存消耗
  push_to_hub=False, #是否将模型发布到hf社区
  report_to='tensorboard', #日志的报告地
  load_best_model_at_end=True, #是否自动加载最优模型
  ignore_data_skip=True #从断点继续训练模型
)
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=image_processor,
    
)


In [14]:
metrics = trainer.evaluate(dataset['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)


NameError: name 'np' is not defined